# Overview
An implementation of the WikiDiverse entity linking system using attention mechanisms, thereby leveraging a multimodal attention-based approach to fuse both text and image information for entity disambiguation.

# 1. Required Libraries 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel, CLIPProcessor, CLIPModel
import torch.nn.functional as F
import PIL
from PIL import Image
from tqdm import tqdm

# 2. Textual Encoder (BERT-based) and Image Encoder (CLIP)

In [ ]:
# Initialize BERT tokenizer and model for textual encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Initialize CLIP processor and model for image encoding
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")

# 3. Cross-Attention Layer

In [ ]:
class CrossAttentionLayer(nn.Module):
    def __init__(self, hidden_size, num_attention_heads):
        super(CrossAttentionLayer, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=num_attention_heads)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, text_features, image_features):
        # Cross-attention: text to image and image to text
        text_features = text_features.unsqueeze(0)  
        image_features = image_features.unsqueeze(0)

        # Perform attention between text and image features
        attn_output_text, _ = self.attention(text_features, image_features, image_features)
        attn_output_image, _ = self.attention(image_features, text_features, text_features)
        
        # Combine the outputs (you can experiment with different strategies like sum, concat, etc.)
        combined_output = attn_output_text + attn_output_image
        combined_output = self.fc(combined_output)  
        return combined_output

# 4. Entity Disambiguation Head

In [ ]:
class EntityDisambiguationHead(nn.Module):
    def __init__(self, hidden_size, num_candidates):
        super(EntityDisambiguationHead, self).__init__()
        self.fc1 = nn.Linear(hidden_size, 512)
        self.fc2 = nn.Linear(512, num_candidates)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, features):
        x = F.relu(self.fc1(features))
        x = self.fc2(x)
        return self.softmax(x)

# 4. Multimodal Entity Linking Model

In [ ]:
class MultimodalEntityLinkingModel(nn.Module):
    def __init__(self, hidden_size, num_attention_heads, num_candidates):
        super(MultimodalEntityLinkingModel, self).__init__()
        self.text_encoder = bert_model
        self.image_encoder = clip_model
        self.cross_attention_layer = CrossAttentionLayer(hidden_size, num_attention_heads)
        self.disambiguation_head = EntityDisambiguationHead(hidden_size, num_candidates)

    def forward(self, text_input, image_input):
        # Textual feature extraction using BERT
        encoded_input = tokenizer(text_input, return_tensors='pt', padding=True, truncation=True)
        text_output = self.text_encoder(**encoded_input).last_hidden_state  # shape: (batch_size, seq_len, hidden_size)
        
        # Image feature extraction using CLIP
        inputs = clip_processor(text=[text_input], images=image_input, return_tensors="pt", padding=True)
        outputs = self.image_encoder(**inputs)
        image_features = outputs.image_embeds  # shape: (batch_size, embedding_dim)

        # Apply cross-attention to fuse text and image features
        combined_features = self.cross_attention_layer(text_output, image_features)

        # Disambiguate and predict the entity
        entity_scores = self.disambiguation_head(combined_features.squeeze(0))  # Removing batch dimension
        return entity_scores

# 5. Help function for calculating Loss and accuracy 

In [ ]:
def calculate_loss_and_accuracy(model, text_input, image_input, correct_entity_index):
    # Forward pass through the model
    entity_scores = model(text_input, image_input)

    # Compute the cross-entropy loss for entity disambiguation
    labels = torch.tensor([correct_entity_index])  # The index of the correct entity in the candidate list
    criterion = nn.CrossEntropyLoss()
    loss = criterion(entity_scores, labels)
    
    # Get the predicted entity
    predicted_entity = torch.argmax(entity_scores, dim=1)
    accuracy = (predicted_entity == labels).float().mean()
    
    return loss, accuracy

# 6. Iniitialize Model and Run 

In [ ]:
# Initialize model parameters
hidden_size = 768  # BERT hidden size
num_attention_heads = 8
num_candidates = 10  # Number of candidates for entity linking

# Initialize the model
model = MultimodalEntityLinkingModel(hidden_size, num_attention_heads, num_candidates)

# Input (text and image) ???
# text_input = "The Lions versus the Packers (2007)."
# image_path = "path_to_image.jpg"
image_input = Image.open(image_path)

# Assume we have the index of the correct entity (for example purposes)
correct_entity_index = 0

# Calculate loss and accuracy
loss, accuracy = calculate_loss_and_accuracy(model, text_input, image_input, correct_entity_index)
print(f"Loss: {loss.item()}, Accuracy: {accuracy.item()}")